# Submit geometry optimization

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# General imports.
import ipywidgets as ipw
from IPython.display import clear_output

# AiiDA imports.
%load_ext aiida
%aiida
# AiiDAlab imports.
import aiidalab_widgets_base as awb
from aiida import orm, plugins

from surfaces_tools.utils import wfn

# Custom imports.
from surfaces_tools.widgets import cdxml, computational_resources, editors, inputs

In [ ]:
Cp2kBenchmarkWorkChain = plugins.WorkflowFactory("nanotech_empa.cp2k.benchmark")

In [ ]:
# Structure selector.
structure_selector = awb.StructureManagerWidget(
    importers=[
        awb.StructureUploadWidget(title="Import from computer"),
        awb.StructureBrowserWidget(title="AiiDA database"),
    ],
    editors=[
        
    ],
    storable=True,
    node_class="StructureData",
)
display(structure_selector)

# Code.
code_input_widget = awb.ComputationalResourcesWidget(
    description="CP2K code:", default_calc_job_plugin="cp2k"
)
#resources = computational_resources.ProcessResourcesWidget()

output = ipw.Output()

In [ ]:
# Protocol.
protocol = ipw.Dropdown(
    value="scf_ot_no_wfn",
    options=[
        "scf_ot_no_wfn"
    ],
    description="Protocol:",
    style={"description_width": "initial"},
)
# List number of nodes.
num_nodes = ipw.Text(
    value='4 8 12 16 24',
    description="List of possible #nodes:",
    style={"description_width": "initial"},
)
# Number of gpus.
num_gpus = ipw.IntText(
    value=1,
    description="Number of GPUs:",
    style={"description_width": "initial"},
)
# List number tasks per node.
max_tasks_per_node = ipw.IntText(
    value=2,
    description="Max #tasks per node:",
    style={"description_width": "initial"},
)
# List number of threads.
num_threads_per_task = ipw.Text(
    value='1 2 4 8',
    description="List of possible #threads per task:",
    style={"description_width": "initial"},
)
# Wallclock minutes.
wallclock_minutes = ipw.IntText(
    value=30,
    description="Wallclock minutes:",
    style={"description_width": "initial"},
)
# Multiplicity.
multiplicity = ipw.IntText(
    value=0,
    description="Multiplicity:",
    style={"description_width": "initial"},
)

In [ ]:
workflow_description = ipw.Text(
    description="Workflow description:",
    placeholder="Provide the description here.",
    style={"description_width": "initial"},
    layout={"width": "70%"},
)

In [ ]:
def prepare_submission():
    msg = ""
    with output:
        clear_output()
    if not structure_selector.structure_node:
        can_submit, msg = False, "Select a structure first."
    elif not code_input_widget.value:
        can_submit, msg = False, "Select CP2K code."
    else:
        can_submit = True

    if not can_submit:
        with output:
            print(msg)
            return

    builder = Cp2kBenchmarkWorkChain.get_builder()
    builder.protocol = orm.Str(protocol.value)
    builder.multiplicity = orm.Int(multiplicity.value)
    builder.metadata.label = "CP2K_benchmark"
    builder.metadata.description = workflow_description.value
    code = orm.load_code(code_input_widget.value)
    builder.code = code
    builder.list_nodes = orm.List(list=[int(i) for i in num_nodes.value.split()])
    builder.max_tasks_per_node = orm.Int(max_tasks_per_node.value)
    builder.ngpus = orm.Int(num_gpus.value)
    builder.list_threads_per_task = orm.List(list=[int(i) for i in num_threads_per_task.value.split()])
    builder.wallclock = orm.Int(wallclock_minutes.value * 60)

    builder.structure = structure_selector.structure_node

    return builder

In [ ]:
btn_submit = awb.SubmitButtonWidget(
    Cp2kBenchmarkWorkChain,
    inputs_generator=prepare_submission,
    disable_after_submit=True,
    append_output=True,
)

# Inputs

In [ ]:

display(ipw.VBox([protocol,multiplicity, num_nodes, num_gpus,max_tasks_per_node, num_threads_per_task, wallclock_minutes]))

# Code and resources

In [ ]:
display(code_input_widget)

# Submit

In [ ]:
display(workflow_description, btn_submit, output)